In [ ]:
%pip install openai pandas --upgrade

In [1]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
from src.genAIClient import GenerativeAIClient
import time
import random

In [2]:
client = GenerativeAIClient(show_info_log=False)

In [82]:
# Define two variants of the prompt
prompt_A = """Product description: A pair of shoes that can fit any foot size.
Themes: adaptable, fit, omni-fit.
Product names:

# output format:
comma separated list."""

prompt_B = """Product description: A home milkshake maker.
Themes: fast, healthy, compact, flexible.
Product names: HomeShaker, Fit Shaker, QuickShake, Shake Maker

Product description: A watch that can tell accurate time in space.
Themes: astronaut, space-hardened, eliptical orbit, outer space.
Product names: AstroTime, SpaceGuard, Orbit-Accurate, EliptoTime.

Product description: A pair of shoes that can fit any foot size.
Themes: adaptable, fit, omni-fit.
Product names:

# output format:
comma separated list."""

def delay(ll_delay=1, ul_delay=2):
    return round(random.uniform(ll_delay, ul_delay), 2)

In [83]:
output = client.invoke(prompt_A)
print(output)

Adapt-All Shoes, Omni-Fit Footwear, Fit-Any-Foot Footwear


In [84]:
# Iterate through the prompts and get responses
test_prompts = [prompt_A, prompt_B]
responses = []
num_tests_per_prompt = 6
max_retries = 3

for idx, prompt in enumerate(test_prompts):
    # prompt number as a letter
    var_name = chr(ord('A') + idx)

    for i in range(num_tests_per_prompt):
        # Get a response from the model
        response = None
        retries = 0
        while response is None and retries < max_retries:
            response = client.invoke(prompt)
            # print(response)
            if response is None:
                seconds = delay()
                print(f"{var_name}: {i}: delaying for {seconds} seconds...")
                time.sleep(seconds)
                retries += 1
                continue
            data = {
                "variant": var_name,
                "prompt": prompt,
                "response": response
                }
            responses.append(data)
            # print(data)


# Convert responses into a DataFrame
df = pd.DataFrame(responses)


In [86]:
df

,variant,prompt,response
0,A,Product description: A pair of shoes that can ...,"Omni-Fit Shoes, Adaptable Footwear"
1,A,Product description: A pair of shoes that can ...,"Omni-Fit Shoes, Adaptable Footwear, Fit-All Shoes"
2,A,Product description: A pair of shoes that can ...,"Omni-Fit Shoes, Adaptable Shoes, Fit-All Shoes"
3,A,Product description: A pair of shoes that can ...,"Omni-Fit, Adapta-Shoes, One-Size-Fits-All Shoe..."
4,A,Product description: A pair of shoes that can ...,"Omni-Fit, AdaptaFit, FitForAll"
5,A,Product description: A pair of shoes that can ...,"Omni-Fit Shoes, AdaptableFit Shoes, FitAnyFoot..."
6,B,Product description: A home milkshake maker.\n...,"AdaptiveShoe, Fit-All, OmniFoot, UniversalShoe..."
7,B,Product description: A home milkshake maker.\n...,"FlexFit, Fit-for-All, OmniShoez"
8,B,Product description: A home milkshake maker.\n...,"Omni-Fit, FitShoe, AdaptaShoe, Fit-All"
9,B,Product description: A home milkshake maker.\n...,"OmniFit, FitTech, Adapti-Shoe, SizeFlex"


------------------------------------------------------

In [77]:
# Shuffle the DataFrame
df = df.sample(frac=1).reset_index(drop=True)

# Initialize variables
response_index = 0
df["feedback"] = pd.Series(dtype="str")  # Add a feedback column

response = widgets.HTML()
count_label = widgets.Label()

def update_response():
    """Update the displayed response and counter."""
    if response_index < len(df):
        new_response = df.iloc[response_index]["response"]
        new_response = f"<p>{new_response}</p>" if pd.notna(new_response) else "<p>No response</p>"
        response.value = new_response
        count_label.value = f"Response: {response_index + 1} / {len(df)}"
    else:
        response.value = "<p>All responses reviewed.</p>"
        count_label.value = f"Response: {response_index} / {len(df)}"

def on_button_clicked(b):
    global response_index

    if response_index < len(df):
        user_feedback = 1 if b.description == "👍" else 0
        df.at[response_index, "feedback"] = user_feedback
        response_index += 1
        print(f"Clicked! response_index is now {response_index}")

        if response_index < len(df):
            update_response()


In [78]:
update_response()

# Create buttons
thumbs_down_button = widgets.Button(description="👎")
thumbs_down_button.on_click(on_button_clicked)

thumbs_up_button = widgets.Button(description="👍")
thumbs_up_button.on_click(on_button_clicked)

# Arrange buttons
button_box = widgets.HBox([thumbs_up_button, thumbs_down_button])

# Display UI elements
display(response, button_box, count_label)

HTML(value='<p>ShapeFit, AdaptShoe, OmniShoe, FitPerfect</p>')

Label(value='Response: 1 / 12')

In [79]:
df

,variant,prompt,response,feedback
0,B,Product description: A home milkshake maker.\n...,"ShapeFit, AdaptShoe, OmniShoe, FitPerfect",1
1,A,Product description: A pair of shoes that can ...,"Omni-Fit Shoes, Fit-All Footwear, Adaptable Fo...",1
2,A,Product description: A pair of shoes that can ...,"Omni-Fit Shoes, Adapt-to-Fit Footwear, Univers...",0
3,B,Product description: A home milkshake maker.\n...,"OmniFit, AnyFit, InstaFit, AdaptoShoe",0
4,B,Product description: A home milkshake maker.\n...,"FitStep, OmniFit, SoleSolution",1
5,A,Product description: A pair of shoes that can ...,"Omni-Fit-Shoes, Adapta-Shoes",0
6,A,Product description: A pair of shoes that can ...,"Adapta-Fit, Omni-Fit, Universal-Fit",1
7,B,Product description: A home milkshake maker.\n...,"OmniFit, Adapt-a-Size, FitFlex",0
8,B,Product description: A home milkshake maker.\n...,"AdaptaFit, OmniFit, ShapeShift, Fit-All, FreeSize",0
9,A,Product description: A pair of shoes that can ...,"Omni-Fit Shoes, All-Fit Footwear, Adapt-A-Soles",1


In [80]:
# Save the DataFrame as a CSV file
csv_file = "../data/responses.csv"
df.to_csv(csv_file, index=False)

In [81]:
print("A/B testing completed. Here are the results:")

if "variant" in df.columns:
    summary_df = df.groupby("variant").agg(
        count=("feedback", "count"),
        score=("feedback", "mean")
    ).reset_index()

    display(summary_df)
    print("Summary displayed!")  # DEBUG
else:
    print("No 'variant' column found. Summary cannot be generated.")

A/B testing completed. Here are the results:


,variant,count,score
0,A,6,0.5
1,B,6,0.5


Summary displayed!
